# Computational Theory Problems

In [1]:
#Library to work with arrays and mathematical functions
import numpy as np


## Probelm 1 Binary Words and Operations

This section implements the 32-bit word primitives used by SHA-256. All operations conceptually act on unsigned 32-bit words. In Python, integers are unbounded, so either mask with `0xFFFFFFFF` to keep values in 32 bits, or use [`numpy.uint32`](https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32) for natural 32-bit wraparound.

Operations and definitions:
- Logical right shift: SHR^n(x) — shift right by n bits, zero-fill on the left.
- Rotate right: ROTR^n(x) — shift right by n bits, wrapping the bits that fall off back to the left.
- Choose: Ch(x, y, z) = (x AND y) XOR ((NOT x) AND z) — per bit, pick y when x’s bit is 1, else z.
- Majority: Maj(x, y, z) = (x AND y) XOR (x AND z) XOR (y AND z) — per bit, returns the majority among x, y, z.
- Big sigmas:
  - Σ0(x) = ROTR^2(x) XOR ROTR^13(x) XOR ROTR^22(x)
  - Σ1(x) = ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)
- Small sigmas:
  - σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)
  - σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)


In [2]:
def ROTR(x, n):
    """
    Rotate right (circular right shift) on a 32-bit word.

    Parameters
    ----------
    x : int
        Value to rotate (treated as an unsigned 32-bit word).
    n : int
        Number of bits to rotate right by.

    Returns
    -------
    numpy.uint32
        x rotated right by n bits (mod 2^32).

    Notes
    -----
    - Uses `numpy.uint32` to enforce 32-bit wraparound semantics.
    - Handles n=0 safely (avoids shifting by 32).
    """
    x = np.uint32(x)
    n = int(n) & 31
    if n == 0:
        return x
    return np.uint32((x >> n) | (x << (32 - n)))

In [3]:
def Parity(x, y, z):
    """
    Bitwise parity (XOR) of three 32-bit words.

    This function is included for completeness (it appears in SHA-1 style boolean
    functions), but SHA-256 primarily uses `Ch` and `Maj`.
    """
    return np.uint32(x) ^ np.uint32(y) ^ np.uint32(z)

In [4]:
def Ch(x, y, z):
    """
    SHA-256 choose function.

    For each bit position i:
    - if x_i = 1, output y_i
    - if x_i = 0, output z_i

    Implemented as: (x AND y) XOR ((NOT x) AND z) over 32-bit words.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

In [5]:
def Maj(x, y, z):
    """
    SHA-256 majority function.

    For each bit position i, the output bit is 1 if at least two of (x_i, y_i, z_i)
    are 1; otherwise it is 0.

    Implemented as: (x AND y) XOR (x AND z) XOR (y AND z) over 32-bit words.
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (x & z) ^ (y & z)

In [6]:
def Sigma0(x):
    """
    SHA256 Sigma0 function
    For a 32-bit integer x, performs right rotations by 2, 13, and 22 bits
    and returns the XOR of the results.
    """
    return (ROTR(x, 2)) ^ (ROTR(x, 13)) ^ (ROTR(x, 22))

In [7]:
def Sigma1(x):
    """
    SHA256 Sigma1 function
    For a 32-bit integer x, performs right rotations by 6, 11 and 25 bits
    and returns the XOR of the results.
    """

    return (ROTR(x, 6)) ^ (ROTR(x, 11)) ^ (ROTR(x, 25))

In [8]:
def sigma0(x):
    """
    SHA-256 small sigma0: σ0(x) = ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x).
    """
    x = np.uint32(x)
    return ROTR(x, 7) ^ ROTR(x, 18) ^ (x >> np.uint32(3))

In [9]:
def sigma1(x):
    """
    SHA-256 small sigma1: σ1(x) = ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x).
    """
    x = np.uint32(x)
    return ROTR(x, 17) ^ ROTR(x, 19) ^ (x >> np.uint32(10))

## Problem 2 Fractional Parts of Cube Roots

Goal: Derive 64 constant 32‑bit words from the fractional parts of the cube roots of the first 64 primes. In SHA‑256 these become the constants K[0..63] (FIPS 180‑4, Section 4.2.2).

Process implemented:
1. Generate the first 64 primes starting at 2.
2. For each prime p compute its real cube root c = p ** (1/3).
3. Separate fractional part: frac = c - int(c).
4. Scale to 32 bits: frac32 = floor(frac * 2**32). This captures the leading 32 bits of the fractional part.

Why multiply by 2**32:
- The spec defines each word as the first 32 bits of the fractional part, multiplying by 2^32 and truncating shifts those bits into the integer portion.


Reference:
- FIPS PUB 180‑4 Secure Hash Standard: https://csrc.nist.gov/publications/detail/fips/180/4/final

In [10]:
#https://stackoverflow.com/questions/54543956/finding-prime-number-using-the-square-root-method
#code adapted from above link
def is_prime(num):
    """
    Check if a number is prime

    This functions checks if a given number is prime,
    it divides the number by integers between 2 and 1 more than its square root
    as if its not divisible by a factor smaller than the square root than it cant be divisible by a larger factor than the square root
    if it passes this test then it is prime
    """
    if num >= 2:
        for i in range(2, int(np.sqrt(num)) + 1):
            if (num % i) == 0:
                return False
    return True


In [11]:
def primes(n):
    """
    Generate the first n prime numbers
    start from 2 and check each number if it is prime using the is_prime function
    if it is prime add it to the list of primes
    add 1 to the number and repeat until we have n primes
    """
    
    primes = []
    num = 2  
    while len(primes) < n:
        if is_prime(num):
            primes.append(num)
        num += 1

    return primes

In [12]:
def frac32():
    """
    Calculate the 32-bit fractional parts of the cube roots of the first 64 prime numbers
    1. Get the first 64 prime numbers using the primes function
    2. For each prime number, calculate its cube root
    3. Isolate the fractional part of the cube root by subtracting the integer part
    4. Convert the fractional part to a 32-bit integer by multiplying by 2^32 and taking the integer part
    5. Store the 32-bit integers in a list and return the list
    """
    #Get the first 64 prime numbers
    prime_numbers = primes(64)
    #List to hold the 32-bit fractional parts of the cube roots
    frac32 = []

    for prime in prime_numbers:
        #Calculate the cube root of the prime number
        cube_root = prime ** (1/3)
        #Int fun is used here to isolate the fractional part from the cube root by removing the integer part
        frac = cube_root - int(cube_root)
        #Convert the fractional part to a 32-bit integer and removing the left over fractional part
        frac32.append(int(frac * (2**32)))
        
    return frac32

In [29]:
for frac in frac32():
    print(f"{frac:08x}")

428a2f98
71374491
b5c0fbcf
e9b5dba5
3956c25b
59f111f1
923f82a4
ab1c5ed5
d807aa98
12835b01
243185be
550c7dc3
72be5d74
80deb1fe
9bdc06a7
c19bf174
e49b69c1
efbe4786
0fc19dc6
240ca1cc
2de92c6f
4a7484aa
5cb0a9dc
76f988da
983e5152
a831c66d
b00327c8
bf597fc7
c6e00bf3
d5a79147
06ca6351
14292967
27b70a85
2e1b2138
4d2c6dfc
53380d13
650a7354
766a0abb
81c2c92e
92722c85
a2bfe8a1
a81a664b
c24b8b70
c76c51a3
d192e819
d6990624
f40e3585
106aa070
19a4c116
1e376c08
2748774c
34b0bcb5
391c0cb3
4ed8aa4a
5b9cca4f
682e6ff3
748f82ee
78a5636f
84c87814
8cc70208
90befffa
a4506ceb
bef9a3f7
c67178f2


## Problem 3 Padding

Goal: Transform a byte message into one or more 512-bit (64-byte) blocks suitable for SHA-256 processing, applying the padding rules from FIPS PUB 180-4 Sections 5.1.1 (Message Padding) and 5.2.1 (Parsing the Message).

Padding steps (before splitting into blocks):
1. Let L = original message length in bits (len(msg) * 8).
2. Append a single '1' bit. In bytes this is 0x80 (binary 10000000).
3. Append k zero bytes (0x00) until (message_length_in_bytes + 1 + k) ≡ 56 (mod 64). Reason: reserve final 8 bytes (64 bits) for L.
4. Append L as a 64-bit unsigned big-endian integer.
5. The padded result length is a multiple of 64 bytes; split into consecutive 64-byte blocks.

In [30]:
def read_file(filepath):
    """
    Read the contents of a file and return it as a byte object.
    """
    with open(filepath, 'rb') as file:
        bytes = file.read()
    return bytes


In [13]:
def block_parse(msg):
    """
    Yield 512-bit (64-byte) blocks after applying SHA-256 padding.

    This implements the message padding and parsing rules in FIPS PUB 180-4:
    - Section 5.1.1 (Message Padding)
    - Section 5.2.1 (Parsing the Message)

    Parameters
    ----------
    msg : bytes
        Input message.

    Yields
    ------
    bytes
        The next 64-byte block (including padding in the final one or two blocks).

    References
    ----------
    - FIPS PUB 180-4: https://csrc.nist.gov/publications/detail/fips/180/4/final
    - Generator pattern: https://realpython.com/introduction-to-python-generators/
    """
    padded = bytearray(msg)
    padded.append(0x80)  # append the single '1' bit (as 10000000)

    # Pad with zeros until length ≡ 56 (mod 64) so we can append the 64-bit length.
    while (len(padded) % 64) != 56:
        padded.append(0x00)

    msg_len_bits = len(msg) * 8
    padded += msg_len_bits.to_bytes(8, byteorder='big')

    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i + 64])

In [32]:
#msg = read_file('README.md')
msg = b'jhgfchgchgf'
for block in block_parse(msg):
    print(block.hex())

6a686766636867636867668000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000058


## Problem 4: Hashes (SHA-256 Compression Function)

Goal: Implement the SHA-256 *compression function* that updates the hash state using one 512-bit message block, as specified in FIPS PUB 180-4 Section 6.2.2.

Inputs and outputs:
- Input `current`: the current hash state as 8 unsigned 32-bit words (H0..H7).
- Input `block`: the next 512-bit message block (64 bytes).
- Output: the next hash state as 8 unsigned 32-bit words.

Algorithm overview (Section 6.2.2):
1. **Prepare the message schedule**
   - Parse `block` into 16 big-endian 32-bit words W[0..15].
   - Extend to W[0..63] using the small sigma functions:
     - W[t] = σ1(W[t−2]) + W[t−7] + σ0(W[t−15]) + W[t−16]  (mod 2^32)
2. **Initialize working variables**
   - Set (a, b, c, d, e, f, g, h) = current state (H0..H7).
3. **Perform 64 rounds**
   - For each round t = 0..63 compute:
     - T1 = h + Σ1(e) + Ch(e,f,g) + K[t] + W[t]  (mod 2^32)
     - T2 = Σ0(a) + Maj(a,b,c)                   (mod 2^32)
   - Update the working variables (a..h) using T1 and T2 (per the standard).
4. **Compute the next hash state**
   - Add the working variables back into the incoming state:
     - H0' = H0 + a, H1' = H1 + b, ..., H7' = H7 + h  (mod 2^32)
   - The resulting 8 words are the new state.

Implementation notes:
- All additions are modulo $2^{32}$ (32-bit wraparound).
- The per-round constants K[0..63] come from Problem 2 (FIPS 180-4 Section 4.2.2).
- The initial hash state H0..H7 is fixed by the standard (FIPS 180-4 Section 5.3.3).
- This compression step is applied once per 64-byte block produced by `block_parse` (Problem 3).

Reference:
- FIPS PUB 180-4 (Secure Hash Standard), Sections 4.2.2, 5.3.3, 6.2.2: https://csrc.nist.gov/publications/detail/fips/180/4/final

In [14]:
# Round constants K[0..63] (FIPS 180-4 Section 4.2.2).
# We reuse Problem 2's computation of the fractional cube-root constants.
K = np.array(frac32(), dtype=np.uint32)

# Initial hash values H0..H7 (FIPS 180-4 Section 5.3.3).
H0 = np.array(
    [
        0x6A09E667, 0xBB67AE85, 0x3C6EF372, 0xA54FF53A,
        0x510E527F, 0x9B05688C, 0x1F83D9AB, 0x5BE0CD19,
    ],
    dtype=np.uint32,
)

In [17]:
def hash(current, block):
    """
    Compute the next SHA-256 hash state from the current state and one 512-bit block.

    This is the SHA-256 compression function defined in FIPS PUB 180-4, Section 6.2.2.

    Parameters
    ----------
    current : iterable[int]
        The current hash state as 8 32-bit words (H0..H7).
    block : bytes
        The next 512-bit message block (exactly 64 bytes).

    Returns
    -------
    list[int]
        The next hash state as 8 unsigned 32-bit integers (as Python ints).

    Notes
    -----
    The name `hash` matches the assignment prompt, but it shadows Python's built-in
    `hash()` function inside this notebook.
    """
    if len(block) != 64:
        raise ValueError("block must be exactly 64 bytes")

    H = np.array(list(current), dtype=np.uint32)
    if H.shape != (8,):
        raise ValueError("current must contain exactly 8 words")

    # Numpy will overflow in uint32 arithmetic by design; suppress overflow warnings.
    with np.errstate(over='ignore'):
        # 1) Message schedule W[0..63]
        W = np.zeros(64, dtype=np.uint32)
        for t in range(16):
            W[t] = np.uint32(int.from_bytes(block[4 * t: 4 * t + 4], "big"))
        for t in range(16, 64):
            W[t] = np.uint32(sigma1(W[t - 2]) + W[t - 7] + sigma0(W[t - 15]) + W[t - 16])

        # 2) Initialize working variables a..h
        a, b, c, d, e, f, g, h = H

        # 3) 64 rounds
        for t in range(64):
            T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
            T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
            h = g
            g = f
            f = e
            e = np.uint32(d + T1)
            d = c
            c = b
            b = a
            a = np.uint32(T1 + T2)

        # 4) Compute the next hash value
        out = np.array([a, b, c, d, e, f, g, h], dtype=np.uint32)
        out = np.uint32(H + out)

    return [int(x) for x in out]

In [18]:
# Minimal end-to-end test: compare against Python's hashlib for a few messages.
import hashlib

def sha256_digest(msg: bytes) -> bytes:
    state = H0
    for blk in block_parse(msg):
        state = np.array(hash(state, blk), dtype=np.uint32)
    return b"".join(int(x).to_bytes(4, "big") for x in state)

for m in [b"", b"abc", b"a" * 55, b"a" * 56, b"a" * 100]:
    ours = sha256_digest(m).hex()
    ref = hashlib.sha256(m).hexdigest()
    print(len(m), ours == ref)

0 True
3 True
55 True
56 True
100 True


## Problem 5 Passwords

## End